In [ ]:
import torch
import torch.nn as nn
from codes/pytorch_network import UNet,UNetDeep
from codes/RobustDice_BCELoss import CombinedLoss
import torch.optim as optim
from torchsummary import summary
from torchview import draw_graph
from codes/slice_loader import SliceLoader
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import time 
from codes/utils import epoch_time
from torch.utils.tensorboard import SummaryWriter

In [2]:
def dice_scoreBCE(pred, target, class_index, threshold=0.6, eps=1e-8):
    try:
        if pred.max() > 1.0 or pred.min() < 0.0:
            pred = torch.sigmoid(pred)

        pred_cls = pred[:, class_index, :, : ,:]
        target_cls = target[:, class_index, : , : , :]

        if pred_cls.shape != target_cls.shape:
            print(f"Shape mismatch: pred {pred_cls.shape}, target {target_cls.shape}")
            return torch.tensor(0.0, device=pred.device)

        intersection = torch.logical_and(pred_cls.bool(), target_cls.bool()).sum(dtype=torch.float32)
        union = pred_cls.sum() + target_cls.sum()

        dice = (2.0 * intersection + eps) / (union + eps)
        return dice

    except Exception as e:
        print(f"Dice computation failed: {e}")
        return torch.tensor(0.0, device=pred.device)

def train_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0.0
    total_dice_nucleus = 0.0
    total_dice_mito = 0.0
    batch_idx = 0
    pbar = tqdm(dataloader, desc="Training")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)

        # pred = torch.argmax(outputs[0], dim=1) 
        pred = torch.sigmoid(outputs)
        pred = (pred>0.6).float()
        dice_1 = dice_scoreBCE(pred, labels, 0) # Nucleus
        dice_2 = dice_scoreBCE(pred, labels, 1) # Mito

        total_dice_nucleus += dice_1.item()
        total_dice_mito += dice_2.item()

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        batch_idx += 1
        pbar.set_postfix({
            'Loss': total_loss / batch_idx,
            'Dice Nucleus': total_dice_nucleus / batch_idx,
            'Dice Mito': total_dice_mito / batch_idx
        })
        pbar.refresh()
        del outputs, inputs, labels
    pbar.close()
    return total_loss / len(dataloader)

def train_epoch_with_accumulation(model, dataloader, loss_fn, optimizer, device, accumulation_steps=2, beta = 0.85):
    model.train()
    total_loss = 0.0
    total_dice_nucleus = 0.0
    total_dice_mito = 0.0
    
    pbar = tqdm(dataloader, desc="Training")
    
    # 1. Zero gradients before the accumulation cycle begins
    optimizer.zero_grad()
    
    for i, (inputs, labels) in enumerate(pbar):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate metrics as before
        with torch.no_grad(): # Metrics don't need to track gradients
            pred = torch.sigmoid(outputs)
            pred = (pred > 0.6).float()
            dice_1 = dice_scoreBCE(pred, labels, 0) # Nucleus
            dice_2 = dice_scoreBCE(pred, labels, 1) # Mito
            total_dice_nucleus += dice_1.item()
            total_dice_mito += dice_2.item()

        # 2. Scale the loss by the number of accumulation steps
        with torch.no_grad():
            probs = torch.sigmoid(outputs)
        Soft_labels = beta*labels + (1-beta)*probs
        loss = loss_fn(outputs, Soft_labels)
        loss = loss / accumulation_steps
        
        # 3. Accumulate the gradients
        loss.backward()

        # 4. Update model weights and zero gradients after N steps or on the last batch
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(dataloader):
            optimizer.step()
            optimizer.zero_grad()
            
        # For logging, un-scale the loss to get the correct batch loss
        total_loss += loss.item() * accumulation_steps
        
        pbar.set_postfix({
            'Loss': total_loss / (i + 1),
            'Dice Nucleus': total_dice_nucleus / (i + 1),
            'Dice Mito': total_dice_mito / (i + 1)
        })
        pbar.refresh()
        del outputs, inputs, labels, loss, pred # Clean up memory
        
    pbar.close()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0.0
    total_dice_nucleus = 0.0
    total_dice_mito = 0.0
    batch_idx = 0
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Evaluating")
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)

            # pred = torch.argmax(outputs[0], dim=1)
            pred = torch.sigmoid(outputs)
            pred = (pred>0.6).float()
            dice_1 = dice_scoreBCE(pred, labels, 0) # Nucleus
            dice_2 = dice_scoreBCE(pred, labels, 1) # Mito

            total_dice_nucleus += dice_1.item()
            total_dice_mito += dice_2.item()

            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            
            batch_idx += 1
            pbar.set_postfix({
                'Loss': total_loss / batch_idx,
                'Val Dice Nucleus': total_dice_nucleus / batch_idx,
                'Val Dice Mito': total_dice_mito / batch_idx
            })
            pbar.refresh()
            del outputs, inputs, labels
        pbar.close()

    return total_loss / len(dataloader), total_dice_nucleus / len(dataloader), total_dice_mito / len(dataloader)
 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Create UNet mode
model = UNet(input_shape = (1,64,704,704), num_classes=2)  

# Define the loss function and optimizer
loss_fn = CombinedLoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)

# Parallelize the model for multi-GPU training
model = nn.DataParallel(model)  
model = model.to(device)

# Load train and val datasets using SliceLoader
train_dataset = SliceLoader("slices/", "Mrc_trainSlices", "Mask_trainSlices") 
val_dataset = SliceLoader("slices/", "Mrc_valSlices", "Mask_valSlices")

# Define DataLoader for training and validating
batchSize = 4
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False)

cuda


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs --port 6006

In [ ]:
import logging
import os

checkpoint_path = "Soft_bootstrap_Model"
progress_path = checkpoint_path + "_progress.pth"
start_epoch = 0
best_valid_loss = float('inf')

if os.path.exists(progress_path):
    progress = torch.load(progress_path)
    start_epoch = progress['epoch'] + 1

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # loss_fn.load_state_dict(checkpoint['loss_fn_state_dict'])
    best_valid_loss = checkpoint['best_valid_loss']
    best_epoch = checkpoint['epoch']+1
    
    print(f"Resuming from epoch {start_epoch}, with best validation loss: {best_valid_loss:.3f} registered at {best_epoch}th epoch")
    
num_epochs = start_epoch + 60
    
writer = SummaryWriter(log_dir="runs/Soft_bootstrap_Model")

train_losses = []
valid_losses = []

logging.basicConfig(filename = 'Soft_bootstrap_Model.log',
                    filemode = 'a',
                    format = '%(asctime)s  [%(levelname)s]  %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level = logging.INFO)
logging.info(f'Starting training for: {checkpoint_path}.\n For Mito+Nucleus Classes.')

for epoch in range(start_epoch, num_epochs):
    start_time = time.time()
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train_epoch_with_accumulation(model, train_loader, loss_fn, optimizer, device, accumulation_steps=2)
    valid_loss, NucDice, MitoDice = evaluate_model(model, val_loader, loss_fn, device)

    logging.info(f'Epoch {epoch}: Train Loss: {train_loss:.3f}, Validation Loss: {valid_loss:.3f}')

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    if valid_loss < best_valid_loss:
        data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
        print(data_str)

        best_valid_loss = valid_loss
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            # 'loss_fn_state_dict': loss_fn.state_dict(),
            'best_valid_loss': best_valid_loss
        }
        torch.save(checkpoint, checkpoint_path)
        
    torch.save({'epoch': epoch}, progress_path)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')
    print(f'\t Nucelus DiceScore: {NucDice:.3f}')
    print(f'\t Mito DiceScore: {MitoDice:.3f}')
    
    torch.cuda.empty_cache()
    
    writer.add_scalar('Train Loss', train_loss, epoch)
    writer.add_scalar('Validation Loss', valid_loss, epoch)
    writer.add_scalar('NucleusDice', NucDice, epoch)
    writer.add_scalar('MitoDice', MitoDice, epoch)
    
    del train_loss,valid_loss,NucDice,MitoDice
# Close the TensorBoard writer
writer.close()